In [1]:
import pandas as pd
import numpy as np
import cv2
from random import randint
import shutil
import os

In [2]:
# load data
data = pd.read_csv('gt.txt', sep=";", header=None)
data.columns = ["img", "x1", "y1", "x2", "y2", "id"]
data.head()

,img,x1,y1,x2,y2,id
0,00000.ppm,774,411,815,446,11
1,00001.ppm,983,388,1024,432,40
2,00001.ppm,386,494,442,552,38
3,00001.ppm,973,335,1031,390,13
4,00002.ppm,892,476,1006,592,39


In [3]:
# define the path of images
imgPath = 'yolo/darknet/sign_data/images/'
# open an image to get the shape
image = cv2.imread(imgPath + '00001.ppm')
shape = image.shape[:2]

### Convert annotation files into YOLO format

In [5]:
def convert_data_to_yolo(df, shape):
    """
    the function convert coordinates into YOLO format. the bounding box of YOLO (x, y, w, h)
    Then normalise the coordinates to be between (0 & 1) 
    by dividing by the real image width & height
    """
    #crete emplty dictionary where keys are image name
    yolo_dic = {i.split('.')[0]: [] for i in df['img']}
    
    for i in range(len(data)):
        img = df.iloc[i]['img'].split('.')[0]

        x = ((df.iloc[i]['x1'] + df.iloc[i]['x2']) / 2.0) / shape[1]  
        y = ((df.iloc[i]['y1'] + df.iloc[i]['y2']) / 2.0 ) / shape[0] 
        w = (df.iloc[i]['x2'] - df.iloc[i]['x1']) / shape[1]
        h = (df.iloc[i]['y2'] - df.iloc[i]['y1']) / shape[0]

        yolo_dic[img].append([x, y, w, h])
        
    return yolo_dic

In [6]:
yolo_dic = convert_data_to_yolo(data, shape)

#### Check point

In [ ]:
# check point
# format is [x, y, w, h]
yolo_pts = yolo_dic['00001']

image = cv2.imread(imgPath + '00001.ppm')
for i in yolo_pts:
    startX = int((i[0] - i[2]/2) * shape[1])
    startY = int((i[1] - i[3]/2) * shape[0])
    endX = int((i[0] + i[2]/2) * shape[1])
    endY = int((i[1] + i[3]/2) * shape[0])
    
    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 150, 100), 5)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# Save the annotations for each image in a seperate file in the same folder of the image
def save_txt_file(path, data_dic):
    for key, value in data_dic.items():
        filePath = path + key + '.txt'
        
        # create an empty list to store sign annotation inside
        text = []
        # save yolo annotation for each sign and store it in a list 
        for v in value:
            line = "{} {} {} {} {}".format(0, v[0], v[1], v[2], v[3])
            text.append(line)
            
            # Write file
            with open(filePath, 'w') as file:
                for item in text:
                    file.writelines("%s\n" % item)

In [9]:
save_txt_file(imgPath, yolo_dic)

#### Check point

In [ ]:
# check that all images have 2 txt files

import collections

ck_files = []
for file in os.listdir(imgPath):
    ck_files.append(file.split('.')[0])
    
counts = collections.Counter(ck_files)
#counts.most_common()

### Split images

In [86]:
# split data into train, test dataset in ratio 90:10
# get image names    
images = list(yolo_dic.keys())

# no of test images
test_size = int(len(yolo_dic) * 10 /100)

# create an empty lists to store test & train images in
test_images = []
train_images = []

# create an empty list to store index
index = []
n = 0
while n < test_size:
    # generate random integer number
    randN = randint(0, len(yolo_dic))
    # check that the no is not chosen before
    if randN not in index:
        index.append(randN)
        n += 1

# add selected indices to test images list
for i in index:
    test_images.append(images[i]+'.ppm')

# add the remaining images to train images list
for i in range(len(yolo_dic)):
    if i not in index:
        train_images.append(images[i]+'.ppm')

In [87]:
print('No of train images: ', len(train_images))
print('No of test images', len(test_images))
print('Total images: ', len(yolo_dic))

No of train images:  667
No of test images 74
Total images:  741


### Save text files for train and test datase

In [89]:
# YOLO needs the path of each image in the both dataset

# save the path of the training images, and of the 
# test images into relevant list
path = 'sign_data/images/'
train_path = [path + img for img in train_images]
test_path = [path + img for img in test_images]

#Create training.txt file
file = open("yolo/darknet/sign_data/train.txt", "w") 
file.write("\n".join(train_path)) 
file.close() 

#Create test.txt file
file = open("yolo/darknet/sign_data/test.txt", "w") 
file.write("\n".join(test_path)) 
file.close() 

### copy annotation files into labels directory

In [85]:
dest = 'yolo/darknet/sign_data/labels/'
for file in os.listdir(imgPath):
    if file.endswith('.txt'):
        shutil.copyfile(imgPath + file, dest+file)

### Creating object labels & data file

In [91]:
#Create sign.names file for label names
file = open("yolo/darknet/sign_data/sign.names", "w") 
file.write("sign") 
file.close() 

In [101]:
# Create sign.data file, it will contain 
#info about classes, training & test datases paths

txt = """classes = 1
train = sign_data/train.txt
valid = sign_data/test.txt
names = sign_data/sign.names
backup = backup
"""
#Create sign.data file for label names
file = open("yolo/darknet/sign_data/sign.data", "w")
for item in txt.split('\n'):
    file.writelines("%s\n" % item)
file.close() 